## Интерактивный чат-бот Phi 3 Mini 4K Instruct с Whisper

### Введение:
Интерактивный чат-бот Phi 3 Mini 4K Instruct — это инструмент, который позволяет пользователям взаимодействовать с демонстрацией Microsoft Phi 3 Mini 4K Instruct с помощью текстового или голосового ввода. Чат-бот может использоваться для выполнения различных задач, таких как перевод, обновления погоды и сбор общей информации.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Создайте ваш токен доступа Huggingface](https://huggingface.co/settings/tokens)

Создайте новый токен  
Укажите новое имя  
Выберите права на запись  
Скопируйте токен и сохраните его в надежном месте


Следующий код на Python выполняет две основные задачи: импорт модуля `os` и установку переменной окружения.

1. Импорт модуля `os`:
   - Модуль `os` в Python предоставляет возможность взаимодействовать с операционной системой. Он позволяет выполнять различные задачи, связанные с операционной системой, такие как доступ к переменным окружения, работа с файлами и директориями и т.д.
   - В этом коде модуль `os` импортируется с помощью оператора `import`. Этот оператор делает функциональность модуля `os` доступной для использования в текущем Python-скрипте.

2. Установка переменной окружения:
   - Переменная окружения — это значение, к которому могут обращаться программы, работающие в операционной системе. Это способ хранения настроек конфигурации или другой информации, которая может использоваться несколькими программами.
   - В этом коде новая переменная окружения устанавливается с использованием словаря `os.environ`. Ключ словаря — `'HF_TOKEN'`, а значение присваивается из переменной `HUGGINGFACE_TOKEN`.
   - Переменная `HUGGINGFACE_TOKEN` определяется прямо над этим фрагментом кода и ей присваивается строковое значение `"hf_**************"` с использованием синтаксиса `#@param`. Этот синтаксис часто используется в Jupyter-ноутбуках для ввода данных пользователем и настройки параметров непосредственно в интерфейсе ноутбука.
   - Установив переменную окружения `'HF_TOKEN'`, она становится доступной для других частей программы или других программ, работающих в той же операционной системе.

В целом, этот код импортирует модуль `os` и устанавливает переменную окружения с именем `'HF_TOKEN'` и значением, указанным в переменной `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Этот фрагмент кода определяет функцию под названием clear_output, которая используется для очистки вывода текущей ячейки в Jupyter Notebook или IPython. Давайте разберем код и поймем его функциональность:

Функция clear_output принимает один параметр под названием wait, который является булевым значением. По умолчанию wait установлен в значение False. Этот параметр определяет, должна ли функция ждать появления нового вывода, чтобы заменить существующий, прежде чем очистить его.

Сама функция используется для очистки вывода текущей ячейки. В Jupyter Notebook или IPython, когда ячейка генерирует вывод, например, текст или графические изображения, этот вывод отображается под ячейкой. Функция clear_output позволяет очистить этот вывод.

Реализация функции не предоставлена в данном фрагменте кода, что обозначено многоточием (...). Многоточие представляет собой заполнитель для фактического кода, который выполняет очистку вывода. Реализация функции может включать взаимодействие с API Jupyter Notebook или IPython для удаления существующего вывода из ячейки.

В целом, эта функция предоставляет удобный способ очистки вывода текущей ячейки в Jupyter Notebook или IPython, что упрощает управление и обновление отображаемого вывода во время интерактивных сеансов кодирования.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Выполните преобразование текста в речь (TTS) с использованием сервиса Edge TTS. Давайте рассмотрим реализацию соответствующих функций одну за другой:

1. `calculate_rate_string(input_value)`: Эта функция принимает входное значение и вычисляет строку скорости для голоса TTS. Входное значение представляет желаемую скорость речи, где значение 1 соответствует нормальной скорости. Функция вычисляет строку скорости, вычитая 1 из входного значения, умножая результат на 100 и определяя знак в зависимости от того, больше или равно ли входное значение 1. Функция возвращает строку скорости в формате "{sign}{rate}".

2. `make_chunks(input_text, language)`: Эта функция принимает текст и язык в качестве параметров. Она разбивает текст на части, основываясь на языковых правилах. В данной реализации, если язык — "English", функция делит текст на части по точкам (".") и удаляет любые начальные или конечные пробелы. Затем она добавляет точку к каждой части и возвращает отфильтрованный список частей.

3. `tts_file_name(text)`: Эта функция генерирует имя файла для аудиофайла TTS на основе входного текста. Она выполняет несколько преобразований текста: удаляет завершающую точку (если она есть), преобразует текст в нижний регистр, удаляет начальные и конечные пробелы и заменяет пробелы на подчеркивания. Затем она сокращает текст до максимальной длины в 25 символов (если он длиннее) или использует полный текст, если он пустой. Наконец, она генерирует случайную строку с использованием модуля [`uuid`] и объединяет её с сокращённым текстом, чтобы создать имя файла в формате "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Эта функция объединяет несколько аудиофайлов в один. Она принимает список путей к аудиофайлам и путь для сохранения результата в качестве параметров. Функция инициализирует пустой объект `AudioSegment` под названием [`merged_audio`]. Затем она проходит по каждому пути к аудиофайлу, загружает аудиофайл с помощью метода `AudioSegment.from_file()` из библиотеки `pydub` и добавляет текущий аудиофайл к объекту [`merged_audio`]. В конце она экспортирует объединённый аудиофайл в указанный путь в формате MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Эта функция выполняет операцию TTS с использованием сервиса Edge TTS. Она принимает список текстовых частей, скорость речи, имя голоса и путь для сохранения результата в качестве параметров. Если количество частей больше одной, функция создаёт директорию для хранения отдельных аудиофайлов частей. Затем она проходит по каждой части, формирует команду Edge TTS с использованием функции `calculate_rate_string()`, имени голоса и текста части, и выполняет команду с помощью функции `os.system()`. Если выполнение команды прошло успешно, она добавляет путь к созданному аудиофайлу в список. После обработки всех частей она объединяет отдельные аудиофайлы с помощью функции `merge_audio_files()` и сохраняет объединённый аудиофайл в указанный путь. Если есть только одна часть, она напрямую генерирует команду Edge TTS и сохраняет аудиофайл в указанный путь. В конце она возвращает путь к созданному аудиофайлу.

6. `random_audio_name_generate()`: Эта функция генерирует случайное имя аудиофайла с использованием модуля [`uuid`]. Она генерирует случайный UUID, преобразует его в строку, берёт первые 8 символов, добавляет расширение ".mp3" и возвращает случайное имя аудиофайла.

7. `talk(input_text)`: Эта функция является основной точкой входа для выполнения операции TTS. Она принимает входной текст в качестве параметра. Сначала она проверяет длину входного текста, чтобы определить, является ли он длинным предложением (600 символов или больше). В зависимости от длины и значения переменной `translate_text_flag` она определяет язык и генерирует список текстовых частей с помощью функции `make_chunks()`. Затем она генерирует путь для сохранения аудиофайла с помощью функции `random_audio_name_generate()`. В конце она вызывает функцию `edge_free_tts()` для выполнения операции TTS и возвращает путь к созданному аудиофайлу.

В целом, эти функции работают вместе, чтобы разбить входной текст на части, сгенерировать имя файла для аудиофайла, выполнить операцию TTS с использованием сервиса Edge TTS и объединить отдельные аудиофайлы в один.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Реализация двух функций: convert_to_text и run_text_prompt, а также объявление двух классов: str и Audio.

Функция convert_to_text принимает audio_path в качестве входного параметра и преобразует аудио в текст с использованием модели под названием whisper_model. Сначала функция проверяет, установлен ли флаг gpu в значение True. Если да, то whisper_model используется с определенными параметрами, такими как word_timestamps=True, fp16=True, language='English' и task='translate'. Если флаг gpu установлен в значение False, whisper_model используется с параметром fp16=False. Полученная транскрипция сохраняется в файл с именем 'scan.txt' и возвращается в виде текста.

Функция run_text_prompt принимает сообщение и историю чата в качестве входных данных. Она использует функцию phi_demo для генерации ответа от чат-бота на основе входного сообщения. Сгенерированный ответ затем передается функции talk, которая преобразует ответ в аудиофайл и возвращает путь к файлу. Класс Audio используется для отображения и воспроизведения аудиофайла. Аудио отображается с помощью функции display из модуля IPython.display, а объект Audio создается с параметром autoplay=True, чтобы аудио начинало воспроизводиться автоматически. История чата обновляется входным сообщением и сгенерированным ответом, после чего возвращаются пустая строка и обновленная история чата.

Класс str — это встроенный класс в Python, который представляет последовательность символов. Он предоставляет различные методы для работы со строками, такие как capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill и другие. Эти методы позволяют выполнять операции, такие как поиск, замена, форматирование и манипуляции со строками.

Класс Audio — это пользовательский класс, который представляет аудиообъект. Он используется для создания аудиоплеера в среде Jupyter Notebook. Класс принимает различные параметры, такие как data, filename, url, embed, rate, autoplay и normalize. Параметр data может быть numpy-массивом, списком семплов, строкой, представляющей имя файла или URL, или необработанными данными PCM. Параметр filename используется для указания локального файла, из которого загружаются аудиоданные, а параметр url — для указания URL, с которого загружаются аудиоданные. Параметр embed определяет, должны ли аудиоданные быть встроены с использованием data URI или ссылаться на исходный источник. Параметр rate задает частоту дискретизации аудиоданных. Параметр autoplay определяет, должно ли аудио начинать воспроизводиться автоматически. Параметр normalize указывает, должны ли аудиоданные быть нормализованы (масштабированы) до максимально возможного диапазона. Класс Audio также предоставляет методы, такие как reload для перезагрузки аудиоданных из файла или URL, и атрибуты, такие как src_attr, autoplay_attr и element_id_attr для получения соответствующих атрибутов аудиоэлемента в HTML.

В целом, эти функции и классы используются для преобразования аудио в текст, генерации аудиоответов от чат-бота, а также отображения и воспроизведения аудио в среде Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
